# Phase 1: Data Integration and Preprocessing

This notebook implements the first phase of our deep learning forecasting plan. The objective is to take the raw data from the four separate CSV files, clean them, merge them into a single master dataset, and save the result for the next phase (Feature Engineering).

**Steps:**
1.  **Load Data**: Load all four CSV files into pandas DataFrames.
2.  **Initial Cleaning**: Perform preliminary cleaning on each DataFrame, such as correcting data types, handling obvious duplicates, and dropping columns with no predictive value.
3.  **Merge DataFrames**: Combine the four DataFrames into a single master table, using the timestamp as the primary key.
4.  **Handle Missing Values**: Address NaNs that result from the merge or were present in the original data.
5.  **Finalize and Save**: Set a proper time-series index, sort the data, and save the processed DataFrame to a fast, type-preserving format (`.parquet`).

## 1. Load Data

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Define file paths
input_dir = Path('../input')
processed_dir = Path('../processed')
processed_dir.mkdir(parents=True, exist_ok=True) # Ensure the output directory exists

# Load the datasets
dispatch_df = pd.read_csv(input_dir / 'DISPATCHREGIONSUM.csv')
price_df = pd.read_csv(input_dir / 'TRADINGPRICE.csv')
weather_df = pd.read_csv(input_dir / 'TRAINING_INDEPENDENT_INPUT.csv')
date_dim_df = pd.read_csv(input_dir / 'DATE_DIM.csv')

print("Data loaded successfully.")

Data loaded successfully.


## 2. Initial Cleaning and Preparation

### 2.1 Clean `dispatch_df` and `price_df`
These two files contain the core supply, demand, and price information. We will convert their timestamps to datetime objects and drop the `RUNNO` column, which has only a single value and is not useful for modeling.

In [2]:
# Clean dispatch data
dispatch_df['SETTLEMENTDATE'] = pd.to_datetime(dispatch_df['SETTLEMENTDATE'])
dispatch_df = dispatch_df.drop(columns=['RUNNO'])

# Clean price data
price_df['SETTLEMENTDATE'] = pd.to_datetime(price_df['SETTLEMENTDATE'])
# We also drop PERIODID as it's redundant with the timestamp
price_df = price_df.drop(columns=['RUNNO', 'PERIODID'])

print("Dispatch and Price data cleaned.")

Dispatch and Price data cleaned.


### 2.2 Clean `date_dim_df`
This file contains holiday and day-type information. We need to fix the duplicate holiday column, convert the date column to datetime, and handle the many missing values in the holiday flags, which should be `False` instead of `NaN`.

In [3]:
# Drop the duplicate column identified in profiling
date_dim_df = date_dim_df.drop(columns=['NSW_PUBLIC_HOLIDAY.1'])

# Convert date column to datetime
date_dim_df['ADJ_DATE'] = pd.to_datetime(date_dim_df['ADJ_DATE'])

# Fill NaNs in holiday columns with False and ensure boolean type
holiday_cols = [col for col in date_dim_df.columns if 'HOLIDAY' in col]
for col in holiday_cols:
    # The columns are read as object type with 'True'/'False' strings and NaNs
    date_dim_df[col] = date_dim_df[col].fillna(False).astype(str).str.lower() == 'true'

print("Date Dimension data cleaned.")
date_dim_df.info()

Date Dimension data cleaned.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10958 entries, 0 to 10957
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ADJ_DATE               10958 non-null  datetime64[ns]
 1   DAY_TYPE_QLD           10958 non-null  object        
 2   QLD_PUBLIC_HOLIDAY     10958 non-null  bool          
 3   TAS_PUBLIC_HOLIDAY     10958 non-null  bool          
 4   NSW_PUBLIC_HOLIDAY     10958 non-null  bool          
 5   VIC_PUBLIC_HOLIDAY     10958 non-null  bool          
 6   IS_QLD_SCHOOL_HOLIDAY  10958 non-null  bool          
dtypes: bool(5), datetime64[ns](1), object(1)
memory usage: 224.9+ KB


/var/folders/2g/gmhqyc3x347fffbcnqypjb100000gn/T/ipykernel_6940/2798712556.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  date_dim_df[col] = date_dim_df[col].fillna(False).astype(str).str.lower() == 'true'
/var/folders/2g/gmhqyc3x347fffbcnqypjb100000gn/T/ipykernel_6940/2798712556.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  date_dim_df[col] = date_dim_df[col].fillna(False).astype(str).str.lower() == 'true'
/var/folders/2g/gmhqyc3x347fffbcnqypjb100000gn/T/ipykernel_6940/2798712556.py:11: FutureWarning: Downcasting object dtype arrays on .fil

### 2.3 Clean `weather_df`
The weather data contains many pre-calculated lag features which we will recreate ourselves later. It also has a `PV_POWER` column with over 70% missing values, which we will drop. The `STATION_NAME` is also a single-value column.

In [4]:
weather_df['DATE_TIME_HH'] = pd.to_datetime(weather_df['DATE_TIME_HH'])

# Drop columns that are redundant, have too many NaNs, or will be re-engineered

# Remove PV_POWER from cols_to_drop so it is retained

cols_to_drop = [col for col in weather_df.columns if 'LAG' in col] 
cols_to_drop += ['STATION_NAME', 'YEAR', 'MONTH', 'DAY_TYPE', 'PERIOD_HH']

weather_df = weather_df.drop(columns=cols_to_drop)


# Add a flag for missing PV_POWER values

weather_df['PV_POWER_MISSING'] = weather_df['PV_POWER'].isnull()

print("Weather data cleaned.")

Weather data cleaned.


## 3. Merge DataFrames

In [5]:
# Merge dispatch and price data on their common keys
df = pd.merge(dispatch_df, price_df, on=['SETTLEMENTDATE', 'REGIONID'], how='outer')

# Merge weather data. Weather is not region-specific in the source, so we merge on timestamp only.
# This broadcasts the same weather data across all regions for a given timestamp.
df = pd.merge(df, weather_df, left_on='SETTLEMENTDATE', right_on='DATE_TIME_HH', how='left')
df = df.drop(columns=['DATE_TIME_HH'])

# Merge date dimension data
df['date_only'] = df['SETTLEMENTDATE'].dt.date
date_dim_df['date_only'] = date_dim_df['ADJ_DATE'].dt.date
df = pd.merge(df, date_dim_df, on='date_only', how='left')
df = df.drop(columns=['date_only', 'ADJ_DATE'])

print(f"Merged DataFrame shape: {df.shape}")
df.info()

Merged DataFrame shape: (2099300, 27)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2099300 entries, 0 to 2099299
Data columns (total 27 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   SETTLEMENTDATE          datetime64[ns]
 1   REGIONID                object        
 2   TOTALDEMAND             float64       
 3   AVAILABLEGENERATION     float64       
 4   AVAILABLELOAD           float64       
 5   DISPATCHABLEGENERATION  float64       
 6   DISPATCHABLELOAD        float64       
 7   NETINTERCHANGE          float64       
 8   INITIALSUPPLY           float64       
 9   CLEAREDSUPPLY           float64       
 10  RRP                     float64       
 11  IS_WORKDAY              object        
 12  IS_SCHOOL_HOLIDAY       object        
 13  AIR_TEMP                float64       
 14  HUMIDITY                float64       
 15  DEW_POINT_TEMP          float64       
 16  WIND_SPEED              float64       
 17  PV_POWER

## 4. Handle Missing Values and Finalize

The merges (especially the 'outer' and 'left' joins) may have introduced missing values. We will now set the timestamp as the index, sort the data, and then apply a robust interpolation strategy for the numeric weather features.

In [6]:
# Set and sort the index
df = df.set_index('SETTLEMENTDATE').sort_index()

# The weather data was only for one region. We will forward/backward fill within each region group.
numeric_cols_to_fill = ['AIR_TEMP', 'HUMIDITY', 'DEW_POINT_TEMP', 'WIND_SPEED', 'APPARENT_TEMP']

# We group by region to prevent data from one region leaking into another during interpolation
df[numeric_cols_to_fill] = df.groupby('REGIONID')[numeric_cols_to_fill].transform(
    lambda x: x.interpolate(method='time').fillna(method='ffill').fillna(method='bfill')
)

# Drop any remaining rows with NaNs (could be from outer joins where a match was not found)
df.dropna(inplace=True)

print("Missing values handled. Final DataFrame info:")
df.info()

/var/folders/2g/gmhqyc3x347fffbcnqypjb100000gn/T/ipykernel_6940/1759621296.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lambda x: x.interpolate(method='time').fillna(method='ffill').fillna(method='bfill')
/var/folders/2g/gmhqyc3x347fffbcnqypjb100000gn/T/ipykernel_6940/1759621296.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lambda x: x.interpolate(method='time').fillna(method='ffill').fillna(method='bfill')
/var/folders/2g/gmhqyc3x347fffbcnqypjb100000gn/T/ipykernel_6940/1759621296.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lambda x: x.interpolate(method='time').fillna(method='ffill').fillna(method='bfill')
/var/folders/2g/gmhqyc3x347fffbcnqypjb100000gn/T/ipykernel_6940/1759621296.py:9: FutureWarning: S

Missing values handled. Final DataFrame info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 214495 entries, 2023-01-01 00:00:00 to 2025-06-13 04:00:00
Data columns (total 26 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   REGIONID                214495 non-null  object 
 1   TOTALDEMAND             214495 non-null  float64
 2   AVAILABLEGENERATION     214495 non-null  float64
 3   AVAILABLELOAD           214495 non-null  float64
 4   DISPATCHABLEGENERATION  214495 non-null  float64
 5   DISPATCHABLELOAD        214495 non-null  float64
 6   NETINTERCHANGE          214495 non-null  float64
 7   INITIALSUPPLY           214495 non-null  float64
 8   CLEAREDSUPPLY           214495 non-null  float64
 9   RRP                     214495 non-null  float64
 10  IS_WORKDAY              214495 non-null  object 
 11  IS_SCHOOL_HOLIDAY       214495 non-null  object 
 12  AIR_TEMP                214495 non-null  float64

## 5. Save Processed Data

We will save the final, cleaned DataFrame to a `.parquet` file. Parquet is a columnar storage format that is highly efficient for analytics and preserves data types, which prevents issues when loading the data in the next notebook.

In [8]:
output_path = processed_dir / 'master_dataset.parquet'
df.to_parquet(output_path)

print(f"Processed data saved to: {output_path}")
display(df.head())

Processed data saved to: ../processed/master_dataset.parquet


,REGIONID,TOTALDEMAND,AVAILABLEGENERATION,AVAILABLELOAD,DISPATCHABLEGENERATION,DISPATCHABLELOAD,NETINTERCHANGE,INITIALSUPPLY,CLEAREDSUPPLY,RRP,...,PV_POWER,APPARENT_TEMP,IS_CALCULATED_APP_TEMP,PV_POWER_MISSING,DAY_TYPE_QLD,QLD_PUBLIC_HOLIDAY,TAS_PUBLIC_HOLIDAY,NSW_PUBLIC_HOLIDAY,VIC_PUBLIC_HOLIDAY,IS_QLD_SCHOOL_HOLIDAY
SETTLEMENTDATE,,,,,,,,,,,,,,,,,,,,,
2023-01-01,NSW1,6912.65,11165.60657,57.0,6369.96,0.0,-542.69,6958.41699,6933.16,93.80,...,0.0,23.3,True,False,NWD,False,False,False,False,False
2023-01-01,QLD1,5804.16,8302.43201,75.0,5784.88,0.0,-19.28,5832.15381,5804.20,96.07,...,0.0,23.3,True,False,NWD,False,False,False,False,False
2023-01-01,SA1,1443.28,2893.74605,147.0,1466.12,0.0,22.84,1454.43481,1442.92,82.92,...,0.0,23.3,True,False,NWD,False,False,False,False,False
2023-01-01,TAS1,988.72,2322.76831,0.0,1443.07,0.0,454.35,1005.17883,1006.79,73.45,...,0.0,23.3,True,False,NWD,False,False,False,False,False
2023-01-01,VIC1,4740.34,8592.02719,125.0,4882.03,0.0,141.69,4806.31689,4758.98,84.20,...,0.0,23.3,True,False,NWD,False,False,False,False,False
